In [60]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

In [61]:
bigM = 1000000

In [62]:
# Load data
locations_df = pd.read_csv("C:/Users/Acer/Documents/GitHub/Tabu-Search-for-CVRPTW/inputs/locations.csv")
order_list_df = pd.read_excel('C:/Users/Acer/Documents/GitHub/Tabu-Search-for-CVRPTW/inputs/order_list_1.xlsx')
travel_matrix_df = pd.read_csv('C:/Users/Acer/Documents/GitHub/Tabu-Search-for-CVRPTW/inputs/travel_matrix.csv')
trucks_df = pd.read_csv('C:/Users/Acer/Documents/GitHub/Tabu-Search-for-CVRPTW/inputs/trucks.csv')

In [63]:
Q = sorted(list(set(trucks_df['truck_max_weight'])))

In [64]:
Q = [Q[0]]*10 + [Q[1]]*8 + [Q[2]]*10 + [Q[3]]*4 + [Q[4]]*5

In [65]:
dest1 = list(set(order_list_df['Destination Code']))[:10]

In [66]:
dest = [str(i) for i in dest1]

In [67]:
order_list_df = order_list_df[order_list_df['Destination Code'].isin(dest1)]

In [68]:
order_list_df1 = order_list_df.sort_values(by = 'Destination Code').groupby('Destination Code').sum("Total Weight").reset_index()

In [69]:
locations_df = locations_df[locations_df['location_code'].isin(dest + ['A123'])]

In [70]:
# Convert loading/unloading windows to minutes with explicit format
locations_df['start_minutes'] = pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.hour * 60 + pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.minute
locations_df['end_minutes'] = pd.to_datetime(locations_df['location_loading_unloading_window_end'], format='%H:%M').dt.hour * 60 + pd.to_datetime(locations_df['location_loading_unloading_window_end'], format='%H:%M').dt.minute

In [71]:
customers = locations_df.sort_values(by = 'location_code').iloc[:len(order_list_df1),:]

In [72]:
locations_df2 = locations_df.sort_values(by = 'location_code')

In [73]:
locations_df2

,location_code,trucks_allowed,location_loading_unloading_window_start,location_loading_unloading_window_end,start_minutes,end_minutes
65,10208776,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
45,10212373,['3 tonner Box'],8:00,22:00,480,1320
90,10219625,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
74,10310757,['3 tonner Box'],8:00,22:00,480,1320
55,10366979,['3 tonner Box'],8:00,22:00,480,1320
101,11950159,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
50,12219936,['3 tonner Box'],8:00,22:00,480,1320
129,12231175,['3 tonner Box'],8:00,22:00,480,1320
38,12779571,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
120,13015040,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320


In [74]:
cap_df = dict(zip(trucks_df['truck_type'], trucks_df['truck_max_weight']))

In [75]:
max_veh_access = []
for i in locations_df2.index:
    max_veh_access.append(cap_df[eval(locations_df2['trucks_allowed'][i])[-1]])

In [76]:
max_veh_access = max_veh_access[len(order_list_df1):] + max_veh_access[:len(order_list_df1)]

In [77]:
max_veh_access

[17000, 7000, 2800, 9200, 2800, 2800, 7000, 2800, 2800, 17000, 9200]

In [78]:
depot = locations_df.sort_values(by = 'location_code').iloc[len(order_list_df1):,:]

In [79]:
Nodes = pd.concat([depot,customers], ignore_index = True)

In [80]:
Nodes

,location_code,trucks_allowed,location_loading_unloading_window_start,location_loading_unloading_window_end,start_minutes,end_minutes
0,A123,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,18:00,480,1080
1,10208776,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
2,10212373,['3 tonner Box'],8:00,22:00,480,1320
3,10219625,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
4,10310757,['3 tonner Box'],8:00,22:00,480,1320
5,10366979,['3 tonner Box'],8:00,22:00,480,1320
6,11950159,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
7,12219936,['3 tonner Box'],8:00,22:00,480,1320
8,12231175,['3 tonner Box'],8:00,22:00,480,1320
9,12779571,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320


In [81]:
vehicles = [ k for k in range(0,len(Q))]
customers = [ i for i in range(1,len(Nodes))]
nodes = [ i for i in range(0, len(Nodes))]

In [82]:
sum(Q)

257400

In [83]:
demand = [0] + list(order_list_df1['Total Weight'])

In [84]:
sum(demand)

7035.0

In [85]:
start_time = list(Nodes['start_minutes'])
finish_time = list(Nodes['end_minutes'])

In [86]:
# Constants
service_time_customer = 20  
service_time_depot = 60 

In [87]:
dest2 = ['A123'] + sorted(dest)

In [88]:
dest3 = {} 
for i in range(len(dest2)):
    dest3[dest2[i]] = i

In [89]:
dest3

{'A123': 0,
 '10208776': 1,
 '10212373': 2,
 '10219625': 3,
 '10310757': 4,
 '10366979': 5,
 '11950159': 6,
 '12219936': 7,
 '12231175': 8,
 '12779571': 9,
 '13015040': 10}

In [90]:
travel_matrix_df = travel_matrix_df[(travel_matrix_df['source_location_code'].isin(dest + ['A123'])) & (travel_matrix_df['destination_location_code'].isin(dest + ['A123']))]

In [91]:
travel_matrix_df['mapped_source'] = travel_matrix_df['source_location_code'].map(dest3)
travel_matrix_df['mapped_destination'] = travel_matrix_df['destination_location_code'].map(dest3)

In [92]:
dist_matrix = {}
time_matrix = {}
for i in travel_matrix_df.index:
    dist_matrix[(travel_matrix_df['mapped_source'][i], travel_matrix_df['mapped_destination'][i])] = travel_matrix_df['travel_distance_in_km'][i]
    time_matrix[(travel_matrix_df['mapped_source'][i], travel_matrix_df['mapped_destination'][i])] = travel_matrix_df['travel_time_in_min'][i]

In [93]:
start_time

[480, 480, 480, 480, 480, 480, 480, 480, 480, 480, 480]

In [94]:
finish_time

[1080, 1320, 1320, 1320, 1320, 1320, 1320, 1320, 1320, 1320, 1320]

In [95]:
nodes

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [96]:
customers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [97]:
vehicles

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36]

In [98]:
#Decision Variable

my_model=gp.Model('CVRPTW')

xijk=my_model.addVars(nodes, nodes, vehicles, vtype=GRB.BINARY, name='xijk')
sik = my_model.addVars(nodes, vehicles, vtype = GRB.CONTINUOUS, name = 'sik',lb = 0)
q = my_model.addVars(customers, vehicles, vtype = GRB.CONTINUOUS, name = 'qjk', lb = 0)
I = my_model.addVars(vehicles, vtype = GRB.BINARY, name = 'I')

In [99]:
#Objective function
obj_fn = (gp.quicksum(dist_matrix[i,j]* gp.quicksum(xijk[i,j,k] for k in vehicles) for i in nodes for j in nodes) +
          gp.quicksum(I[k] for k in vehicles)
         )

my_model.setObjective(obj_fn, GRB.MINIMIZE)


In [100]:
#Source to sink constraints
my_model.addConstrs(gp.quicksum(xijk[0,j,k] for j in customers)<=1 for k in vehicles);

In [101]:
my_model.addConstrs(gp.quicksum(xijk[i,0,k] for i in customers)<=1 for k in vehicles);

In [102]:
# my_model.addConstr(gp.quicksum(xijk[0,j,k] for j in customers for k in vehicles)<=10);

In [103]:
my_model.addConstrs(gp.quicksum(xijk[i,j,k] for i in nodes)- gp.quicksum(xijk[j,i,k] for i in nodes)==0 
                    for j in nodes for k in vehicles);

In [104]:
#capacity constraints
my_model.addConstrs(gp.quicksum(q[j,k] for k in vehicles) == demand[j] 
                    for j in customers); 
my_model.addConstrs(gp.quicksum(q[j,k] for j in customers) <= Q[k] for k in vehicles);

In [105]:
my_model.addConstrs(q[j,k] <= Q[k] * gp.quicksum(xijk[i,j,k] for i in nodes) for j in customers for k in vehicles);

In [106]:
my_model.addConstrs(sik[i,k] + time_matrix[i,j] + service_time_customer
                    - sik[j,k] <= (1-xijk[i,j,k]) *1440
                    for i in customers 
                    for j in customers for k in vehicles);

In [107]:
my_model.addConstrs(sik[i,k] <= finish_time[i] for i in nodes for k in vehicles)
                    
my_model.addConstrs(sik[i,k] >= start_time[i] for i in nodes for k in vehicles);                                   

In [108]:
for i in nodes:
    for j in nodes:
        for k in vehicles:
            if Q[k] <= max_veh_access[j] and Q[k] <= max_veh_access[i]:
                my_model.addConstr(xijk[i,j,k] <= 1);
            else:
                my_model.addConstr(xijk[i,j,k] == 0);

In [109]:
my_model.addConstrs(gp.quicksum(xijk[0,j,k] for j in customers) <= I[k] for k in vehicles);

In [110]:
my_model.setParam('Heuristics', 0.6)  

Set parameter Heuristics to value 0.6


In [52]:
my_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9926 rows, 5291 columns and 30118 nonzeros
Model fingerprint: 0xfa510fb2
Variable types: 777 continuous, 4514 integer (4514 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 8109 rows and 3179 columns
Presolve time: 0.04s
Presolved: 1817 rows, 2112 columns, 9697 nonzeros
Variable types: 409 continuous, 1703 integer (1703 binary)
Found heuristic solution: objective 961.1400000

Root relaxation: objective 2.672600e+02, 413 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\ipykernel\iostream.py", line 655, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 20403  2055  365.68392   28   14  384.52000  354.73117  7.75%  15.7   60s
 22176  1988 infeasible   31       384.52000  357.02474  7.15%  15.8   65s


In [ ]:
# Create a new dictionary to store non-zero values
non_zero_xijk = {}

# Iterate over all keys and values in xijk
for (i, j, k), value in xijk.items():
    # Check if the value is a Gurobi variable
    if hasattr(value, "X"):  # If it's a Gurobi variable, use .X to get the optimized value
        if value.X > 0.5:  # Filter for non-zero values
            non_zero_xijk[(i, j, k)] = value.X
    else:  # Otherwise, assume it's a numeric value
        if value > 0.5:  # Filter for non-zero values
            non_zero_xijk[(i, j, k)] = value

# Print the non-zero values
print("Non-zero values of x:")
for (i, j, k), value in non_zero_xijk.items():
    print(f"x_{i}_{j}_{k} = {value}")


Non-zero values of x:
x_0_3_8 = 1.0
x_1_5_8 = 1.0
x_2_0_8 = 1.0
x_3_4_8 = 1.0
x_4_1_8 = 1.0
x_5_2_8 = 1.0


In [54]:
# Create a dictionary to store routes for each vehicle
vehicle_routes = {}

# Iterate over the non_zero_xijk dictionary to create routes
for (i, j, k), value in non_zero_xijk.items():
    if value == 1.0:
        if k not in vehicle_routes:
            vehicle_routes[k] = []  # Create a list for each vehicle if not already created
        vehicle_routes[k].append((i, j))

# Display the routes for each vehicle
for vehicle, route in vehicle_routes.items():
    print(f"Vehicle {vehicle} Route: {route}")

Vehicle 8 Route: [(0, 3), (1, 5), (2, 0), (3, 4), (4, 1), (5, 2)]


In [55]:
max_veh_access

[17000, 7000, 2800, 2800, 2800, 9200]

In [56]:
non_zero_q = {(i,k):q[i,k].X for i in customers for k in vehicles if q[i,k].X > 0.1}
non_zero_q

{(1, 8): 611.0, (2, 8): 209.0, (3, 8): 304.0, (4, 8): 84.0, (5, 8): 220.0}

In [57]:
demand

[0, 611.0, 209.0, 304.0, 84.0, 220.0]

In [58]:
I

{0: <gurobi.Var I[0] (value 0.0)>,
 1: <gurobi.Var I[1] (value 0.0)>,
 2: <gurobi.Var I[2] (value 0.0)>,
 3: <gurobi.Var I[3] (value 0.0)>,
 4: <gurobi.Var I[4] (value 0.0)>,
 5: <gurobi.Var I[5] (value 0.0)>,
 6: <gurobi.Var I[6] (value 0.0)>,
 7: <gurobi.Var I[7] (value 0.0)>,
 8: <gurobi.Var I[8] (value 1.0)>,
 9: <gurobi.Var I[9] (value 0.0)>,
 10: <gurobi.Var I[10] (value 0.0)>,
 11: <gurobi.Var I[11] (value 0.0)>,
 12: <gurobi.Var I[12] (value 0.0)>,
 13: <gurobi.Var I[13] (value 0.0)>,
 14: <gurobi.Var I[14] (value 0.0)>,
 15: <gurobi.Var I[15] (value 0.0)>,
 16: <gurobi.Var I[16] (value 0.0)>,
 17: <gurobi.Var I[17] (value 0.0)>,
 18: <gurobi.Var I[18] (value 0.0)>,
 19: <gurobi.Var I[19] (value 0.0)>,
 20: <gurobi.Var I[20] (value 0.0)>,
 21: <gurobi.Var I[21] (value 0.0)>,
 22: <gurobi.Var I[22] (value 0.0)>,
 23: <gurobi.Var I[23] (value 0.0)>,
 24: <gurobi.Var I[24] (value 0.0)>,
 25: <gurobi.Var I[25] (value 0.0)>,
 26: <gurobi.Var I[26] (value 0.0)>,
 27: <gurobi.Var I[27

In [59]:
#use dest3 for reverse mapping into location codes from index